In [ ]:
#download pySpark

!java -version

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version

!pip install pyspark

!pip install lifetimes

In [ ]:
import pandas as pd
import numpy as np
import os
import datetime, time
from pyspark.sql.types import StructType, StructField, StringType
import pyspark.sql.functions as F
from pyspark import SparkContext, SparkConf

from pyspark.sql.functions import unix_timestamp, col, to_date, struct, abs,when, round, desc, concat, countDistinct

from pyspark.sql import Row
from pyspark.sql.functions import sum as _sum
from pyspark.sql.functions import max as max_

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("App_Name") \
    .config("spark.driver.memory", "64g") \
    .getOrCreate()

In [ ]:
#data load

from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
path1 = "/content/drive/MyDrive/NDSU_capstone/2.Raw_data/acc1"
path2 = "/content/drive/MyDrive/NDSU_capstone/2.Raw_data/acc2"

In [ ]:
### Read all files in Transaction folder
df1 = spark.read.csv(path1,header=True)

df2 = spark.read.csv(path2,header=True)

In [ ]:
df = df1.union(df2)
df1.count()
df2.count()
df.count()

34068363

In [ ]:
# df1.count()

In [ ]:
# df2.count()

In [ ]:
df.show()

+--------------------+-----------+--------------+----------+-----------+-------+------------+----------+-----------------+-------------------+-------------+--------------------+
|         account_key|ACTIVE_DATE|account_status| closed_dt|eom_prod_dt|balance|orig_bal_amt|major_type|major_description|  minor_description|customer_type|       Household_key|
+--------------------+-----------+--------------+----------+-----------+-------+------------+----------+-----------------+-------------------+-------------+--------------------+
|4fd13b2b71798b555...| 2011-06-09|           CLS|2018-12-03| 2021-12-31|    0.0|    224000.0|       MTG|    Mortgage Loan|  Team Member Loans|     Personal|34c6b044a52962f00...|
|4fd13b2b71798b555...| 2011-06-09|           CLS|2018-12-03| 2022-10-31|    0.0|    224000.0|       MTG|    Mortgage Loan|  Team Member Loans|     Personal|34c6b044a52962f00...|
|4fd13b2b71798b555...| 2011-06-09|           CLS|2018-12-03| 2022-05-31|    0.0|    224000.0|       MTG|    Mo

In [ ]:
# df_test = df.filter(df.account_key  == "297a9e99b5b8799153d0869b43d5aa62")

In [ ]:
# df_test.sort(col("eom_prod_dt")).show(76)

In [ ]:
# df_test2 = df.filter(df.account_key  == "e64662981ff1c83dab8954059d31ba52")
# df_test2.sort(col("eom_prod_dt")).show(df_test2.count())

In [ ]:
# df_test3 = df.filter(df.account_key  == "5f0191ccea701038db6b6bad532ed846")
# df_test3.sort(col("eom_prod_dt")).show(df_test2.count())

In [ ]:
# df_equal.coalesce(1).write.csv('/content/drive/MyDrive/NDSU_capstone/3.proceeded_data/testing_2.csv')

In [ ]:
df_eom = df.filter(df.eom_prod_dt.isin(["2021-01-31","2020-12-31"]))

df_eom.count()

985929

In [ ]:
# df_eom.coalesce(1).write.csv('/content/drive/MyDrive/NDSU_capstone/3.proceeded_data/testing_eom.csv')

In [ ]:
count_dis = df.groupBy(F.col("account_key")).agg(F.countDistinct(F.col("account_key")))
count_dis.show()

+--------------------+------------------+
|         account_key|count(account_key)|
+--------------------+------------------+
|203ae3bc8cac86fb4...|                 1|
|14528f8257e981a9a...|                 1|
|c852e2b748e6eebec...|                 1|
|b6375c434013ae902...|                 1|
|fafb442e56fd3def5...|                 1|
|55abb9814a178357c...|                 1|
|004fa6a78f72e050f...|                 1|
|a14741f594183c129...|                 1|
|676eca436b4447362...|                 1|
|10ad131abfd853347...|                 1|
|28b89720eae6471aa...|                 1|
|2f144b30df4c15d7e...|                 1|
|f3316a4d31aa216a6...|                 1|
|55116105cc059cb31...|                 1|
|bc31976245adf8320...|                 1|
|ba4b07dc9e900b625...|                 1|
|cc4bc827d79467e98...|                 1|
|eba2925decb851a12...|                 1|
|43b9c609366ff059e...|                 1|
|eda40633fa23b3cc2...|                 1|
+--------------------+------------

In [ ]:
count_dis.count()

698054

In [ ]:
df_ck = df.filter(df.major_type  == "CK")
# df_ck.count()

In [ ]:
df_personal_ck = df_ck.filter(df.customer_type  == "Personal")
df_personal_ck.count()

12889495

In [ ]:
df_personal_ck.write.parquet("/content/drive/MyDrive/NDSU_capstone/3.proceeded_data/acc_all_eom.parquet") 

In [ ]:
count_dis_personalCK = df_personal_ck.groupBy(F.col("account_key")).agg(F.countDistinct(F.col("account_key")))
count_dis_personalCK.count()

271227

In [ ]:
df_date = df_personal_ck.filter(df_personal_ck.eom_prod_dt == "2022-12-31")
df_date.count()

205396

In [ ]:
df_date.coalesce(1).write.csv('/content/drive/MyDrive/NDSU_capstone/3.proceeded_data/result.csv')